In [5]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

In [7]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

31 documents
11 classes ['food', 'goodbye', 'greeting', 'hours', 'location', 'opentoday', 'payments', 'restaurant', 'specials', 'thanks', 'today']
47 unique stemmed words ["'s", 'acceiv', 'any', 'anyon', 'ar', 'best', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'dish', 'do', 'find', 'food', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'loc', 'mastercard', 'of', 'on', 'op', 'resta', 'see', 'sel', 'spec', 'tak', 'thank', 'that', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [9]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00166 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.00166 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.00159 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.00159 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/home/griffin/Documents/ChatBot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:


def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))



In [13]:


p = bow("is your restaurant open today?", words)
print (p)
print (classes)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 1 0 0 0 1 0]
['food', 'goodbye', 'greeting', 'hours', 'location', 'opentoday', 'payments', 'restaurant', 'specials', 'thanks', 'today']


In [14]:
print(model.predict([p]))

[[  1.85777401e-06   5.77824899e-08   2.28459900e-16   1.90006322e-05
    3.08666429e-14   9.30195153e-01   1.13338087e-17   5.13120554e-04
    2.84057666e-10   4.81406093e-15   6.92707226e-02]]


In [15]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )